In [6]:
from scipy.sparse import dok_matrix,csr_matrix
import numpy as np
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 4, 2])
data = np.array([1, 2, 3, 4, 5, 6])
t=csr_matrix((data, (row, col))).toarray()

In [7]:
t.shape

(3, 5)

In [1]:
import numpy as np
import pandas as pd
data_path="../data/ml-100k/u.data"
data_fields = ['user_id', 'item_id', 'rating', 'timestamp']
# all data file
data_df = pd.read_table(data_path, names=data_fields)

In [2]:
K=10
for i in ['user_id', 'item_id']:
    data_df[i] = data_df[i].map(dict(zip(data_df[i].unique(), range(1,data_df[i].nunique()+1))))
    
# get user number
n_users = max(data_df['user_id'].values)
# get item number
n_items = max(data_df['item_id'].values)


In [3]:
from scipy.sparse import dok_matrix,coo_matrix
# S = dok_matrix((5, 5), dtype=np.float32)

data=np.ones((data_df.shape[0]))
row=data_df.user_id-1
col=data_df.item_id-1

UI=coo_matrix((data, (row, col)), shape=(n_users, n_items))

UIU=UI.dot(UI.transpose())
IUI=UI.transpose().dot(UI)

In [43]:
heteSim['UI']

{0: array([0.04444444, 0.02857143, 0.02816901, 0.02816901, 0.02352941,
        0.02325581, 0.02325581, 0.01923077, 0.01709402, 0.01481481]),
 1: array([0.02061856, 0.02040816, 0.02040816, 0.02020202, 0.02020202,
        0.01960784, 0.01869159, 0.01801802, 0.01801802, 0.01666667]),
 2: array([0.01470588, 0.01470588, 0.01449275, 0.01449275, 0.01449275,
        0.0141844 , 0.01408451, 0.01388889, 0.01388889, 0.0137931 ]),
 3: array([0.00833333, 0.00809717, 0.008     , 0.00793651, 0.00790514,
        0.0078125 , 0.0077821 , 0.00769231, 0.00769231, 0.00769231]),
 4: array([0.05128205, 0.03125   , 0.03125   , 0.02247191, 0.01388889,
        0.01369863, 0.01315789, 0.01273885, 0.01111111, 0.01      ]),
 5: array([0.01515152, 0.01298701, 0.01204819, 0.01204819, 0.01176471,
        0.01169591, 0.01086957, 0.0106383 , 0.0104712 , 0.01036269]),
 6: array([0.01769912, 0.01754386, 0.01666667, 0.01587302, 0.01574803,
        0.01550388, 0.01470588, 0.01459854, 0.0141844 , 0.01388889]),
 7: array([0.

In [5]:
from tqdm import tqdm
heteSim={}
heteSimI={}


In [57]:
a

array([[         2, 1072693248],
       [         0, 1072693248]])

In [59]:
np.int

int

In [56]:
a[0][0]=2.5

In [49]:
a=np.empty((2,2),dtype=int)

In [60]:
def calSim(path, M, hinSim, hinSimI):
    hinSim[path]=np.empty((M.shape[0],K))
    hinSimI[path]=np.empty((M.shape[0],K),dtype=np.int)
    M=M.tocsc()
    col_sum={}
    for i in tqdm(range(M.shape[1])):
        col_sum[i]=M.getcol(i).toarray().sum()
    M=M.tocsr()
    row_sum={}
    for i in tqdm(range(M.shape[0])):
        row_sum[i]=M.getrow(i).toarray().sum()
    for i in tqdm(range(M.shape[0])):
        M_i=M.getrow(i).toarray()
        sim=[]
        for j in range(M_i.shape[1]):
            M_ij=M_i[0][j]
            M_i_=row_sum[i] 
            M_j_=col_sum[j]

            S_ij=2*M_ij/(M_i_+M_j_)
            sim.append(S_ij)
        sim=np.array(sim)
        ids=np.argsort(-sim)
        hinSimI[path][i]=ids[:K]
        hinSim[path][i]=sim[ids[:K]]
    
# 使用每次getrow(i) 需要10min 速度太慢了 
# 可以优化 预处理每个的row sum与col sum

In [61]:
calSim('UIU',IUI,heteSim, heteSimI)
calSim('IUI',IUI,heteSim, heteSimI)
calSim('UI',UI,heteSim, heteSimI)

100%|###############################################################################| 943/943 [00:01<00:00, 632.03it/s]


In [64]:
heteSim['UIU'].shape

(1682, 10)

In [18]:
# 单个测试的代码

heteSim['UIU']={}
heteSimI['UIU']={}
UIU=UIU.tocsr()
row_sum={}
for i in tqdm(range(UIU.shape[0])):
    row_sum[i]=UIU.getrow(i).toarray().sum()
UIU=UIU.tocsc()
col_sum={}
for i in tqdm(range(UIU.shape[0])):
    col_sum[i]=UIU.getcol(i).toarray().sum()


for i in tqdm(range(UIU.shape[0])):
    M_i=UIU.getrow(i).toarray()
    sim=[]
    for j in range(M_i.shape[1]):
        M_ij=M_i[0][j]
#         M_i_=row_sum[i] # 这是整个矩阵路径计算 相似度不能为1
#         M_j_=row_sum[j] # 这是整个矩阵路径计算 相似度不能为1
        M_i_=row_sum[i] # 
        M_j_=col_sum[j] #
    
        S_ij=2*M_ij/(M_i_+M_j_)
        sim.append(S_ij)
    sim=np.array(sim)
    s
    ids=np.argsort(-sim)
    heteSimI['UIU'][i]=ids[:K]
    heteSim['UIU'][i]=sim[ids[:K]]
    
# 使用每次getrow(i) 需要10min 速度太慢了 
# 可以优化 预处理每个的row sum与col sum

100%|###############################################################################| 943/943 [00:02<00:00, 372.98it/s]


In [106]:
#### 性能测试 最终使用csr

from tqdm import tqdm
i=0
M_i=UIU.getrow(i).toarray()
sim=[]
for j in tqdm(range(M_i.shape[1])):
    M_ij=M_i[0][j]
    M_i_=UI_csr.getrow(i).toarray().sum()
#     M_j_=UI_csr.getrow(j).toarray().sum()  # 10274
#     M_j_=UI_csc.getcol(j).toarray().sum()  # 6260
    M_j_=UI_csr.getcol(j).toarray().sum()  # 3100
#     M_i_=UI.tocsr().getrow(i).toarray().sum() # 110
#     M_j_=UI.tocsc().getcol(j).toarray().sum()  # 118

    
#     M_j_=UI_csr.transpose().getrow(j).toarray().sum()  2468
#     print(M_ij,M_i_,M_j_)
    S_ij=2*M_ij/(M_i_+M_j_)
    sim.append(S_ij)
#     break
# result = heapq.nlargest(k, range(len(sim)), sim.__getitem__)



# 用coo 42it/s
# 两个都是csr + getrow 10274it/s 但要求同种类型
# 一个csr 一个csr转置 2468it/s
# 一个csr 一个csc  6260.58it/s 但占用两份空间
# 两个都是csr 一个getrow 一个getcol 3100it/s 占用空间较少
# 每个循环都tocsr，其它和上面一行相同 110it/s
# 每个循环分别tocsr tocsc，其它和上面一行相同 118it/s


100%|###############################################################################| 943/943 [00:07<00:00, 118.39it/s]
